# Exploratory Data Analysis (EDA) - IPTU Data

This notebook explores all IPTU (property tax) data files:
- CSV files for years 2020-2023
- JSON file for year 2024

We will load all files, examine their schemas, and perform basic data exploration.


In [2]:
import pandas as pd
import json
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully!")


Libraries imported successfully!


## 1. Load CSV Files (2020-2023)


In [3]:
# Define file paths
csv_files = {
    2020: 'data/iptu_2020/iptu_2020.csv',
    2021: 'data/iptu_2021/iptu_2021.csv',
    2022: 'data/iptu_2022/iptu_2022.csv',
    2023: 'data/iptu_2023/iptu_2023.csv'
}

# Dictionary to store dataframes
dataframes = {}

print("Loading CSV files...")
for year, file_path in csv_files.items():
    print(f"\nLoading {year} data from {file_path}...")
    try:
        # CSV files use semicolon as delimiter
        df = pd.read_csv(file_path, sep=';', encoding='utf-8', low_memory=False)
        dataframes[year] = df
        print(f"✓ Loaded {year}: {len(df):,} rows, {len(df.columns)} columns")
        print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    except Exception as e:
        print(f"✗ Error loading {year}: {str(e)}")

print(f"\n✓ Successfully loaded {len(dataframes)} CSV files")


Loading CSV files...

Loading 2020 data from data/iptu_2020/iptu_2020.csv...
✓ Loaded 2020: 403,915 rows, 32 columns
  Memory usage: 834.43 MB

Loading 2021 data from data/iptu_2021/iptu_2021.csv...
✓ Loaded 2021: 407,245 rows, 32 columns
  Memory usage: 841.34 MB

Loading 2022 data from data/iptu_2022/iptu_2022.csv...
✓ Loaded 2022: 411,090 rows, 32 columns
  Memory usage: 849.32 MB

Loading 2023 data from data/iptu_2023/iptu_2023.csv...
✓ Loaded 2023: 415,029 rows, 32 columns
  Memory usage: 857.52 MB

✓ Successfully loaded 4 CSV files


## 2. Print Schemas for All CSV Files


In [4]:
print("=" * 80)
print("SCHEMAS FOR ALL CSV FILES")
print("=" * 80)

for year in sorted(dataframes.keys()):
    df = dataframes[year]
    print(f"\n{'='*80}")
    print(f"SCHEMA FOR IPTU {year}")
    print(f"{'='*80}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"\nColumn Names and Data Types:")
    print("-" * 80)
    
    schema_info = pd.DataFrame({
        'Column': df.columns,
        'Data Type': df.dtypes,
        'Non-Null Count': df.count(),
        'Null Count': df.isnull().sum(),
        'Null Percentage': (df.isnull().sum() / len(df) * 100).round(2)
    })
    
    print(schema_info.to_string(index=False))
    print(f"\n")


SCHEMAS FOR ALL CSV FILES

SCHEMA FOR IPTU 2020
Shape: 403,915 rows × 32 columns

Column Names and Data Types:
--------------------------------------------------------------------------------
                             Column Data Type  Non-Null Count  Null Count  Null Percentage
             Número do contribuinte    object          403915           0              0.0
                   ano do exercício     int64          403915           0              0.0
              data do cadastramento    object          403915           0              0.0
               tipo de contribuinte    object          403915           0              0.0
 CPF/CNPJ mascarado do contribuinte    object          403915           0              0.0
                         logradouro    object          403915           0              0.0
                             numero     int64          403915           0              0.0
                        complemento    object          403915           0       

## 3. Load JSON File (2024)


In [5]:
json_file_path = 'data/iptu_2024_json/iptu_2024_json.json'

print(f"Loading JSON file: {json_file_path}")
print("=" * 80)

# First, let's read the JSON structure
with open(json_file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Extract schema information
print("\nJSON File Structure:")
print(f"Keys: {list(json_data.keys())}")

if 'fields' in json_data:
    print(f"\nNumber of fields: {len(json_data['fields'])}")
    print("\nField Definitions:")
    print("-" * 80)
    fields_df = pd.DataFrame(json_data['fields'])
    print(fields_df.to_string(index=False))

if 'records' in json_data:
    print(f"\nNumber of records: {len(json_data['records']):,}")

print("\nConverting JSON records to DataFrame...")
# Convert records to DataFrame
if 'records' in json_data and 'fields' in json_data:
    # Get column names from fields
    column_names = [field['id'] for field in json_data['fields']]
    
    # Create DataFrame from records
    df_2024 = pd.DataFrame(json_data['records'], columns=column_names)
    dataframes[2024] = df_2024
    
    print(f"✓ Loaded 2024: {len(df_2024):,} rows, {len(df_2024.columns)} columns")
    print(f"  Memory usage: {df_2024.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


Loading JSON file: data/iptu_2024_json/iptu_2024_json.json

JSON File Structure:
Keys: ['fields', 'records']

Number of fields: 33

Field Definitions:
--------------------------------------------------------------------------------
     type                                  id
      int                                 _id
     text              Número do contribuinte
  numeric                    ano do exercício
timestamp               data do cadastramento
     text                tipo de contribuinte
     text  CPF/CNPJ mascarado do contribuinte
     text                          logradouro
  numeric                              numero
     text                         complemento
     text                              bairro
     text                              cidade
     text                              estado
     text                        fração ideal
     text                        AREA TERRENO
     text                     AREA CONSTRUIDA
     text                       

## 4. Print Schema for JSON File (2024)


In [6]:
if 2024 in dataframes:
    df_2024 = dataframes[2024]
    print("=" * 80)
    print("SCHEMA FOR IPTU 2024 (JSON)")
    print("=" * 80)
    print(f"Shape: {df_2024.shape[0]:,} rows × {df_2024.shape[1]} columns")
    print(f"\nColumn Names and Data Types:")
    print("-" * 80)
    
    schema_info = pd.DataFrame({
        'Column': df_2024.columns,
        'Data Type': df_2024.dtypes,
        'Non-Null Count': df_2024.count(),
        'Null Count': df_2024.isnull().sum(),
        'Null Percentage': (df_2024.isnull().sum() / len(df_2024) * 100).round(2)
    })
    
    print(schema_info.to_string(index=False))
    print(f"\n")


SCHEMA FOR IPTU 2024 (JSON)
Shape: 500 rows × 33 columns

Column Names and Data Types:
--------------------------------------------------------------------------------
                             Column Data Type  Non-Null Count  Null Count  Null Percentage
                                _id     int64             500           0              0.0
             Número do contribuinte    object             500           0              0.0
                   ano do exercício     int64             500           0              0.0
              data do cadastramento    object             500           0              0.0
               tipo de contribuinte    object             500           0              0.0
 CPF/CNPJ mascarado do contribuinte    object             500           0              0.0
                         logradouro    object             500           0              0.0
                             numero     int64             500           0              0.0
             

## 5. Compare Column Names Across All Years


In [7]:
print("=" * 80)
print("COLUMN COMPARISON ACROSS ALL YEARS")
print("=" * 80)

# Get all unique column names
all_columns = set()
for year, df in dataframes.items():
    all_columns.update(df.columns)

all_columns = sorted(all_columns)

# Create comparison DataFrame
comparison_data = {'Column': all_columns}
for year in sorted(dataframes.keys()):
    comparison_data[f'In_{year}'] = [
        '✓' if col in dataframes[year].columns else '✗' 
        for col in all_columns
    ]

comparison_df = pd.DataFrame(comparison_data)
print("\nColumn presence across years:")
print(comparison_df.to_string(index=False))

# Check if all years have the same columns
print("\n" + "=" * 80)
print("Column Consistency Check:")
print("=" * 80)

base_year = sorted(dataframes.keys())[0]
base_columns = set(dataframes[base_year].columns)

all_same = True
for year in sorted(dataframes.keys())[1:]:
    year_columns = set(dataframes[year].columns)
    if base_columns != year_columns:
        all_same = False
        print(f"\n{year} differs from {base_year}:")
        only_in_base = base_columns - year_columns
        only_in_year = year_columns - base_columns
        if only_in_base:
            print(f"  Only in {base_year}: {only_in_base}")
        if only_in_year:
            print(f"  Only in {year}: {only_in_year}")

if all_same:
    print(f"\n✓ All years have the same columns ({len(base_columns)} columns)")


COLUMN COMPARISON ACROSS ALL YEARS

Column presence across years:
                             Column In_2020 In_2021 In_2022 In_2023 In_2024
                    AREA CONSTRUIDA       ✓       ✓       ✓       ✓       ✓
                       AREA TERRENO       ✓       ✓       ✓       ✓       ✓
                                CEP       ✓       ✓       ✓       ✓       ✓
 CPF/CNPJ mascarado do contribuinte       ✓       ✓       ✓       ✓       ✓
                  Código Logradouro       ✓       ✓       ✓       ✓       ✓
             Número do contribuinte       ✓       ✓       ✓       ✓       ✓
       Regime de Tributação da trsd       ✓       ✓       ✓       ✓       ✓
       Regime de Tributação do iptu       ✓       ✓       ✓       ✓       ✓
                 Tipo de Construção       ✓       ✓       ✓       ✓       ✓
             Tipo de Empreendimento       ✓       ✓       ✓       ✓       ✓
                  Tipo de Estrutura       ✓       ✓       ✓       ✓       ✓
                      

## 6. Basic Statistics for Each Dataset


In [8]:
print("=" * 80)
print("BASIC STATISTICS FOR EACH DATASET")
print("=" * 80)

for year in sorted(dataframes.keys()):
    df = dataframes[year]
    print(f"\n{'='*80}")
    print(f"BASIC STATISTICS - IPTU {year}")
    print(f"{'='*80}")
    print(f"\nDataset Overview:")
    print(f"  Total rows: {len(df):,}")
    print(f"  Total columns: {len(df.columns)}")
    print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print(f"\nMissing Values Summary:")
    missing_summary = df.isnull().sum()
    missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
    if len(missing_summary) > 0:
        missing_df = pd.DataFrame({
            'Column': missing_summary.index,
            'Missing Count': missing_summary.values,
            'Missing Percentage': (missing_summary.values / len(df) * 100).round(2)
        })
        print(missing_df.to_string(index=False))
    else:
        print("  ✓ No missing values found")
    
    print(f"\nNumeric Columns Summary:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(df[numeric_cols].describe().to_string())
    else:
        print("  No numeric columns found")
    
    print(f"\nCategorical Columns (first 10):")
    categorical_cols = df.select_dtypes(include=['object']).columns[:10]
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            unique_count = df[col].nunique()
            print(f"\n  {col}:")
            print(f"    Unique values: {unique_count}")
            if unique_count <= 20:
                print(f"    Values: {df[col].value_counts().head(10).to_dict()}")
            else:
                print(f"    Top 5 values: {df[col].value_counts().head(5).to_dict()}")
    print()


BASIC STATISTICS FOR EACH DATASET

BASIC STATISTICS - IPTU 2020

Dataset Overview:
  Total rows: 403,915
  Total columns: 32
  Memory usage: 834.43 MB

Missing Values Summary:
  ✓ No missing values found

Numeric Columns Summary:
       ano do exercício         numero  ano da construção corrigido  quantidade de pavimentos  ano e mês de início da contribuição           CEP  Código Logradouro
count          403915.0  403915.000000                403915.000000             403915.000000                        403915.000000  4.039150e+05      403915.000000
mean             2020.0     541.768474                  1987.060869                  0.565097                        198767.937024  5.113046e+07       36596.305938
std                 0.0     948.889888                    19.393033                  0.825026                          1935.555423  1.925287e+06       36240.840249
min              2020.0      -1.000000                  1088.000000                 -1.000000                     

## 7. Sample Data from Each Dataset


In [9]:
print("=" * 80)
print("SAMPLE DATA FROM EACH DATASET (First 5 rows)")
print("=" * 80)

for year in sorted(dataframes.keys()):
    df = dataframes[year]
    print(f"\n{'='*80}")
    print(f"SAMPLE DATA - IPTU {year}")
    print(f"{'='*80}")
    print(df.head(5).to_string())
    print()


SAMPLE DATA FROM EACH DATASET (First 5 rows)

SAMPLE DATA - IPTU 2020
  Número do contribuinte  ano do exercício          data do cadastramento tipo de contribuinte CPF/CNPJ mascarado do contribuinte                                               logradouro  numero                                                                                           complemento                bairro                                    cidade                          estado fração ideal AREA TERRENO AREA CONSTRUIDA área ocupada valor do m2 do terreno valor do m2 de construção  ano da construção corrigido  quantidade de pavimentos           tipo de uso do imóvel    tipo de padrão da construção fator de obsolescência  ano e mês de início da contribuição valor total do imóvel estimado valor cobrado de IPTU       CEP                        Regime de Tributação do iptu                                                                          Regime de Tributação da trsd              Tipo de Construção      

## 8. Data Quality Checks


In [10]:
print("=" * 80)
print("DATA QUALITY CHECKS")
print("=" * 80)

for year in sorted(dataframes.keys()):
    df = dataframes[year]
    print(f"\n{'='*80}")
    print(f"DATA QUALITY - IPTU {year}")
    print(f"{'='*80}")
    
    # Duplicate rows
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate rows: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")
    
    # Check for potential ID column
    id_cols = [col for col in df.columns if 'id' in col.lower() or '_id' in col.lower()]
    if id_cols:
        print(f"\nPotential ID columns: {id_cols}")
        for col in id_cols:
            unique_count = df[col].nunique()
            print(f"  {col}: {unique_count:,} unique values out of {len(df):,} rows")
    
    # Check date columns
    date_cols = [col for col in df.columns if 'data' in col.lower() or 'ano' in col.lower() or 'timestamp' in col.lower()]
    if date_cols:
        print(f"\nDate/Year columns: {date_cols}")
    
    # Check for empty strings
    empty_strings = {}
    for col in df.select_dtypes(include=['object']).columns:
        empty_count = (df[col] == '').sum()
        if empty_count > 0:
            empty_strings[col] = empty_count
    
    if empty_strings:
        print(f"\nColumns with empty strings:")
        for col, count in sorted(empty_strings.items(), key=lambda x: x[1], reverse=True):
            print(f"  {col}: {count:,} ({count/len(df)*100:.2f}%)")
    
    print()


DATA QUALITY CHECKS

DATA QUALITY - IPTU 2020

Duplicate rows: 60 (0.01%)

Potential ID columns: ['cidade', 'fração ideal', 'AREA CONSTRUIDA', 'ano da construção corrigido', 'quantidade de pavimentos']
  cidade: 1 unique values out of 403,915 rows
  fração ideal: 7,261 unique values out of 403,915 rows
  AREA CONSTRUIDA: 36,912 unique values out of 403,915 rows
  ano da construção corrigido: 106 unique values out of 403,915 rows
  quantidade de pavimentos: 2 unique values out of 403,915 rows

Date/Year columns: ['ano do exercício', 'data do cadastramento', 'ano da construção corrigido', 'ano e mês de início da contribuição']


DATA QUALITY - IPTU 2021

Duplicate rows: 60 (0.01%)

Potential ID columns: ['cidade', 'fração ideal', 'AREA CONSTRUIDA', 'ano da construção corrigido', 'quantidade de pavimentos']
  cidade: 1 unique values out of 407,245 rows
  fração ideal: 7,265 unique values out of 407,245 rows
  AREA CONSTRUIDA: 36,964 unique values out of 407,245 rows
  ano da construção co

## 9. Summary and Next Steps


In [11]:
print("=" * 80)
print("SUMMARY - ALL DATASETS")
print("=" * 80)

summary_data = []
for year in sorted(dataframes.keys()):
    df = dataframes[year]
    summary_data.append({
        'Year': year,
        'Rows': f"{len(df):,}",
        'Columns': len(df.columns),
        'Memory (MB)': f"{df.memory_usage(deep=True).sum() / 1024**2:.2f}",
        'Missing Values': df.isnull().sum().sum(),
        'Duplicate Rows': df.duplicated().sum()
    })

summary_df = pd.DataFrame(summary_data)
print("\nOverview of all datasets:")
print(summary_df.to_string(index=False))

print("\n" + "=" * 80)
print("All schemas have been printed above for detailed review.")
print("=" * 80)


SUMMARY - ALL DATASETS

Overview of all datasets:
 Year    Rows  Columns Memory (MB)  Missing Values  Duplicate Rows
 2020 403,915       32      841.16               0              60
 2021 407,245       32      848.19               0              60
 2022 411,090       32      856.31               0              60
 2023 415,029       32      864.79               0              60
 2024     500       33        1.03               0               0

All schemas have been printed above for detailed review.
